In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
print(selenium.__version__)

4.8.2


In [207]:
options = webdriver.ChromeOptions()
driver_path = '/Users/Jenny/Desktop/gwangjin/chromedriver'
driver = webdriver.Chrome(executable_path=driver_path)

/var/folders/x_/s6rt1hw118zdhq387czgy5q40000gn/T/ipykernel_32530/2377166203.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


## **크롤링 구간**

### **가게 메뉴 및 정보 크롤링** ###

In [208]:
#value = '광진구 화양동 13-4'

#value = '광진구 군자동 149-6'

#value = '광진구 중곡동 245-7'
# value = '광진구 긴고랑로 146'

# value = '광진구 능동로32길 65'

# value = '아차산로73길 48'

# # value = '광진구 천호대로 716'

# value = '광진구 자양로30길 79'

# value = '광진구 뚝섬로 569'

value = '광진구 뚝섬로 493'

In [209]:
url = 'https://www.yogiyo.co.kr/mobile/#/'
driver.get(url)
time.sleep(2)

# 검색창 선택
search_input = driver.find_element(By.XPATH, '//*[@id="search"]/div/form/input')
search_input.click()
search_input.clear()
time.sleep(0.5)

# 검색창 입력
search_input.send_keys(value)
time.sleep(0.5)

# 검색 버튼 클릭
search_button = driver.find_element(By.XPATH, '//*[@id="button_search_address"]/button[2]')
search_button.click()
time.sleep(1.5)

# 최상위 검색값 클릭
click_first = driver.find_element(By.XPATH, '//*[@id="search"]/div/form/ul/li[3]/a')
click_first.click()

In [38]:
# 스크롤 끝까지
interval = 1
prev_height = driver.execute_script("return document.body.scrollHeight")

    # 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height


# 전체 상점 개수 추출
soup = BeautifulSoup(driver.page_source, 'lxml')

store_list = []
stores = driver.find_elements(By.CLASS_NAME, 'col-sm-6')
stores_num = len(list(set(soup.find_all('div', class_='restaurant-name ng-binding'))))
for s in stores:
    store_list.append(s)
stores_num = len(list(set(stores)))
print('stores_num : ', stores_num)

stores_num :  1535


#### 초기 가게정보

In [ ]:
start = 0

store_data = []
end = stores_num
num = start

while num < end:
    try:
        in_store = driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[4]/div/div[2]/div[{num+1}]/div/table/tbody/tr/td[2]')
        driver.execute_script("arguments[0].click();", in_store)
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'lxml')

    except (NoSuchElementException, TimeoutException):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
        )
        continue
    
    address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

    if '광진구' not in address:
        driver.back()
    else:
        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        openhr = soup.find_all('span', class_='tc ng-binding')[1].text.strip()
        mean_rate_div = soup.find('div', class_='restaurant-star-point')
        mean_rate = mean_rate_div.find('strong').text.strip()
        current_url = driver.current_url
        menu_elements = soup.find_all('ul', class_='sub-list')
        
        for parent_element in menu_elements:
            data_rows = parent_element.find_all('div', class_='menu-name ng-binding')
            
            for row in data_rows:
                menu_name = row.text.strip()
                parent_row = row.find_parent('tr')
                photo_exists = 0 if not parent_row or ('ng-hide' in parent_row.get('class', [])) else 1

                store_data.append({
                    'store': store_name,
                    'address': address,
                    'url': current_url, 
                    'openhr': openhr,
                    'menu': menu_name,
                    'photo_exists': photo_exists,
                    'mean_rate': mean_rate
                })
        print(f'{num}. {store_name} 크롤링 완료')
        driver.back()
        time.sleep(0.5)

    num += 1  # 다음 가게로 이동

# DataFrame으로 변환 및 중복 제거
store_df = pd.DataFrame(store_data).drop_duplicates().reset_index(drop=True)
store_df

#### 중복 제거 크롤링

In [70]:
df = pd.read_csv('dataset/store_data_0405.csv', index_col=0)
store_list = df['store'].unique().tolist()

In [71]:
start = 0
store_data = []

end = stores_num
num = start

while num < end:
    try:
        in_store = driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[4]/div/div[2]/div[{num+1}]/div/table/tbody/tr/td[2]')
        driver.execute_script("arguments[0].click();", in_store)
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'lxml')

    except (NoSuchElementException, TimeoutException):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
        )
        continue
    
    address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

    if '광진구' not in address:
        driver.back()
    else:

        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        
        # 여기서 store_name이 이미 크롤링된 리스트에 있는지 확인
        if store_name in store_list:
            print(f'{num}. {store_name}은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.')
            driver.back()
            time.sleep(0.5)
            num += 1
            continue

        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        openhr = soup.find_all('span', class_='tc ng-binding')[1].text.strip()
        mean_rate_div = soup.find('div', class_='restaurant-star-point')
        mean_rate = mean_rate_div.find('strong').text.strip()
        current_url = driver.current_url
        menu_elements = soup.find_all('ul', class_='sub-list')
        
        for parent_element in menu_elements:
            data_rows = parent_element.find_all('div', class_='menu-name ng-binding')
            
            for row in data_rows:
                menu_name = row.text.strip()
                parent_row = row.find_parent('tr')
                photo_exists = 0 if not parent_row or ('ng-hide' in parent_row.get('class', [])) else 1

                store_data.append({
                    'store': store_name,
                    'address': address,
                    'url': current_url, 
                    'openhr': openhr,
                    'menu': menu_name,
                    'photo_exists': photo_exists,
                    'mean_rate': mean_rate
                })
        print(f'{num}. {store_name} 크롤링 완료')
        driver.back()
        time.sleep(0.5)

    num += 1  # 다음 가게로 이동

# DataFrame으로 변환 및 중복 제거
store_df = pd.DataFrame(store_data).drop_duplicates().reset_index(drop=True)
store_df

1. 버거킹-건대입구역점(24)은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
2. 백억커피-화양점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
3. 육초연비빔밥&초밥은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
4. 혼밥에빠진날-광진성수점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
6. 미친피자-건대본점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
7. 파스타예요-광진건대본점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
8. 꽃피는곱도리탕-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
9. 요거트월드-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
10. 춘리마라탕-구의점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
11. 번쩍피자-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
12. 육회바른연어-화양점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
13. 플러스82-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
14. 삼청동김치찜은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
16. 카츠와이찌-중곡점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
17. 봉구가래떡볶이-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
18. 곱도리탕전문점-광진점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
19. 육초연육회초밥연어은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
20. BHC-자양3동점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
21. 지코바치킨-중곡1호점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
22. 남산국밥-광진성수점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
23. 라이라이스볶음밥&돈까스-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
24. 인생BBQ-양키고기-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
25. 홍콩반점0410

KeyboardInterrupt: 

In [72]:
store_df = pd.DataFrame(store_data).drop_duplicates().reset_index(drop=True)
store_df.to_csv('dataset/0405_자양동(2).csv')
store_df

""


### **리뷰 데이터 크롤링**

In [25]:
df = pd.read_csv("dataset/store_data_0405.csv", index_col=0)
url_list = df['url'].unique().tolist()
url_list

['https://www.yogiyo.co.kr/mobile/#/405486/',
 'https://www.yogiyo.co.kr/mobile/#/1028621/',
 'https://www.yogiyo.co.kr/mobile/#/1057967/',
 'https://www.yogiyo.co.kr/mobile/#/1209938/',
 'https://www.yogiyo.co.kr/mobile/#/1323414/',
 'https://www.yogiyo.co.kr/mobile/#/1025675/',
 'https://www.yogiyo.co.kr/mobile/#/1111596/',
 'https://www.yogiyo.co.kr/mobile/#/250260/',
 'https://www.yogiyo.co.kr/mobile/#/1216009/',
 'https://www.yogiyo.co.kr/mobile/#/1274137/',
 'https://www.yogiyo.co.kr/mobile/#/1020159/',
 'https://www.yogiyo.co.kr/mobile/#/1325568/',
 'https://www.yogiyo.co.kr/mobile/#/1294345/',
 'https://www.yogiyo.co.kr/mobile/#/587463/',
 'https://www.yogiyo.co.kr/mobile/#/500097/',
 'https://www.yogiyo.co.kr/mobile/#/516583/',
 'https://www.yogiyo.co.kr/mobile/#/550330/',
 'https://www.yogiyo.co.kr/mobile/#/1011750/',
 'https://www.yogiyo.co.kr/mobile/#/516279/',
 'https://www.yogiyo.co.kr/mobile/#/237173/',
 'https://www.yogiyo.co.kr/mobile/#/287504/',
 'https://www.yogiyo.c

In [26]:
url_list = url_list[2:400]

In [27]:
review_data = []
failed_urls = []

for url in url_list:
    try:
        driver.get(url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'lxml')

        # 가게 기본 데이터
        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

        # 리뷰 데이터
        review_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
        review_btn.click()
        time.sleep(1)

        def collect_reviews():
            soup = BeautifulSoup(driver.page_source, 'lxml')
            review_elements = soup.find_all('li', class_="list-group-item star-point ng-scope")
            for review_element in review_elements:
                score = review_element.find('span', class_='category')
                categorical_score = score.find_all('span', class_='points ng-binding')
                taste_score = categorical_score[0].text.strip()
                quantity_score = categorical_score[1].text.strip()
                review_time = review_element.find('span', class_='review-time ng-binding').text.strip()
                delivery_score = categorical_score[2].text.strip() if len(categorical_score) > 2 else None
                review_content = review_element.find('p', class_='ng-binding').text.strip()

                review_data.append({
                    'store_name': store_name,
                    'address': address,
                    'taste_score': taste_score,
                    'quantity_score': quantity_score,
                    'delivery_score': delivery_score,
                    'review': review_content,
                    'review_time': review_time
                })

        collect_reviews()

        # 더보기 버튼 클릭 함수
        def click_more_button():
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                plus_btn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, plus_btn_xpath))
                )
                plus_btn.click()
                return True
            except:
                return False

        i = 0
        while True:
            i += 1
            var = i * 10 + 2
            plus_btn_xpath = f'/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[{var}]/a'
            collect_reviews()

            if not click_more_button():
                break

            if i % 50 == 0:
                print(f"{i*10}번째 리뷰 크롤링 중...")

            time.sleep(1.5)  # 클릭 후 대기 시간 추가

    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        failed_urls.append(url)
        continue  # 다음 URL로 넘어갑니다.

review_df = pd.DataFrame(review_data).drop_duplicates().reset_index(drop=True)

# 실패한 URL이 있을 경우, 확인 가능
print("Failed URLs:", failed_urls)


500번째 리뷰 크롤링 중...
500번째 리뷰 크롤링 중...
500번째 리뷰 크롤링 중...
1000번째 리뷰 크롤링 중...
1500번째 리뷰 크롤링 중...
500번째 리뷰 크롤링 중...
1000번째 리뷰 크롤링 중...
1500번째 리뷰 크롤링 중...
500번째 리뷰 크롤링 중...
1000번째 리뷰 크롤링 중...
1500번째 리뷰 크롤링 중...
2000번째 리뷰 크롤링 중...
2500번째 리뷰 크롤링 중...
3000번째 리뷰 크롤링 중...
3500번째 리뷰 크롤링 중...
4000번째 리뷰 크롤링 중...
4500번째 리뷰 크롤링 중...
5000번째 리뷰 크롤링 중...
5500번째 리뷰 크롤링 중...
6000번째 리뷰 크롤링 중...
6500번째 리뷰 크롤링 중...
7000번째 리뷰 크롤링 중...
Error while processing https://www.yogiyo.co.kr/mobile/#/250260/: list index out of range
Error while processing https://www.yogiyo.co.kr/mobile/#/1216009/: 'NoneType' object has no attribute 'text'
Error while processing https://www.yogiyo.co.kr/mobile/#/1274137/: 'NoneType' object has no attribute 'text'
500번째 리뷰 크롤링 중...
1000번째 리뷰 크롤링 중...
1500번째 리뷰 크롤링 중...
2000번째 리뷰 크롤링 중...
Error while processing https://www.yogiyo.co.kr/mobile/#/1325568/: 'NoneType' object has no attribute 'text'
500번째 리뷰 크롤링 중...
Error while processing https://www.yogiyo.co.kr/mobile/#/587463/: 'NoneType' 

In [28]:
review_df = pd.DataFrame(review_data).drop_duplicates().reset_index(drop=True)
review_df.to_csv('dataset/review_data_0406(2).csv')

### **실패한 URL** 재시도

In [3]:
failed_urls = ['https://www.yogiyo.co.kr/mobile/#/355621/',
 'https://www.yogiyo.co.kr/mobile/#/549931/',
 'https://www.yogiyo.co.kr/mobile/#/1096018/',
 'https://www.yogiyo.co.kr/mobile/#/1311018/',
 'https://www.yogiyo.co.kr/mobile/#/254572/',
 'https://www.yogiyo.co.kr/mobile/#/1146408/',
 'https://www.yogiyo.co.kr/mobile/#/578487/',
 'https://www.yogiyo.co.kr/mobile/#/1120869/',
 'https://www.yogiyo.co.kr/mobile/#/1208241/',
 'https://www.yogiyo.co.kr/mobile/#/1324311/',
 'https://www.yogiyo.co.kr/mobile/#/1106330/',
 'https://www.yogiyo.co.kr/mobile/#/238752/',
 'https://www.yogiyo.co.kr/mobile/#/2063/',
 'https://www.yogiyo.co.kr/mobile/#/274555/',
 'https://www.yogiyo.co.kr/mobile/#/268302/',
 'https://www.yogiyo.co.kr/mobile/#/1197519/',
 'https://www.yogiyo.co.kr/mobile/#/297266/',
 'https://www.yogiyo.co.kr/mobile/#/1093194/',
 'https://www.yogiyo.co.kr/mobile/#/1050600/',
 'https://www.yogiyo.co.kr/mobile/#/1202475/',
 'https://www.yogiyo.co.kr/mobile/#/530520/',
 'https://www.yogiyo.co.kr/mobile/#/1218303/',
 'https://www.yogiyo.co.kr/mobile/#/1075134/',
 'https://www.yogiyo.co.kr/mobile/#/510243/',
 'https://www.yogiyo.co.kr/mobile/#/254883/',
 'https://www.yogiyo.co.kr/mobile/#/346774/',
 'https://www.yogiyo.co.kr/mobile/#/549089/',
 'https://www.yogiyo.co.kr/mobile/#/1323930/',
 'https://www.yogiyo.co.kr/mobile/#/507305/',
 'https://www.yogiyo.co.kr/mobile/#/1029166/',
 'https://www.yogiyo.co.kr/mobile/#/472721/',
 'https://www.yogiyo.co.kr/mobile/#/1096085/',
 'https://www.yogiyo.co.kr/mobile/#/1085741/',
 'https://www.yogiyo.co.kr/mobile/#/1176856/',
 'https://www.yogiyo.co.kr/mobile/#/1184934/',
 'https://www.yogiyo.co.kr/mobile/#/319827/',
 'https://www.yogiyo.co.kr/mobile/#/1323653/',
 'https://www.yogiyo.co.kr/mobile/#/520369/',
 'https://www.yogiyo.co.kr/mobile/#/443038/',
 'https://www.yogiyo.co.kr/mobile/#/83302/',
 'https://www.yogiyo.co.kr/mobile/#/586014/',
 'https://www.yogiyo.co.kr/mobile/#/425565/',
 'https://www.yogiyo.co.kr/mobile/#/1061927/',
 'https://www.yogiyo.co.kr/mobile/#/24498/',
 'https://www.yogiyo.co.kr/mobile/#/468676/',
 'https://www.yogiyo.co.kr/mobile/#/1323654/',
 'https://www.yogiyo.co.kr/mobile/#/40436/',
 'https://www.yogiyo.co.kr/mobile/#/317294/',
 'https://www.yogiyo.co.kr/mobile/#/322041/',
 'https://www.yogiyo.co.kr/mobile/#/369213/',
 'https://www.yogiyo.co.kr/mobile/#/1189693/',
 'https://www.yogiyo.co.kr/mobile/#/1287950/',
 'https://www.yogiyo.co.kr/mobile/#/1228680/',
 'https://www.yogiyo.co.kr/mobile/#/4597/',
 'https://www.yogiyo.co.kr/mobile/#/1270336/',
 'https://www.yogiyo.co.kr/mobile/#/263183/',
 'https://www.yogiyo.co.kr/mobile/#/474247/',
 'https://www.yogiyo.co.kr/mobile/#/1103570/',
 'https://www.yogiyo.co.kr/mobile/#/249743/',
 'https://www.yogiyo.co.kr/mobile/#/1213364/',
 'https://www.yogiyo.co.kr/mobile/#/580446/',
 'https://www.yogiyo.co.kr/mobile/#/276408/',
 'https://www.yogiyo.co.kr/mobile/#/1081834/',
 'https://www.yogiyo.co.kr/mobile/#/527346/',
 'https://www.yogiyo.co.kr/mobile/#/319449/',
 'https://www.yogiyo.co.kr/mobile/#/1298716/',
 'https://www.yogiyo.co.kr/mobile/#/1295091/',
 'https://www.yogiyo.co.kr/mobile/#/506487/',
 'https://www.yogiyo.co.kr/mobile/#/1137636/',
 'https://www.yogiyo.co.kr/mobile/#/415834/',
 'https://www.yogiyo.co.kr/mobile/#/1321949/',
 'https://www.yogiyo.co.kr/mobile/#/1031136/',
 'https://www.yogiyo.co.kr/mobile/#/370351/',
 'https://www.yogiyo.co.kr/mobile/#/1020358/',
 'https://www.yogiyo.co.kr/mobile/#/1286682/',
 'https://www.yogiyo.co.kr/mobile/#/1217367/',
 'https://www.yogiyo.co.kr/mobile/#/1180412/',
 'https://www.yogiyo.co.kr/mobile/#/1333818/',
 'https://www.yogiyo.co.kr/mobile/#/1171243/',
 'https://www.yogiyo.co.kr/mobile/#/1313934/',
 'https://www.yogiyo.co.kr/mobile/#/231147/',
 'https://www.yogiyo.co.kr/mobile/#/386235/',
 'https://www.yogiyo.co.kr/mobile/#/454778/',
 'https://www.yogiyo.co.kr/mobile/#/251628/',
 'https://www.yogiyo.co.kr/mobile/#/1271730/',
 'https://www.yogiyo.co.kr/mobile/#/520695/',
 'https://www.yogiyo.co.kr/mobile/#/1313347/',
 'https://www.yogiyo.co.kr/mobile/#/1247188/',
 'https://www.yogiyo.co.kr/mobile/#/49325/',
 'https://www.yogiyo.co.kr/mobile/#/1175900/',
 'https://www.yogiyo.co.kr/mobile/#/1029710/',
 'https://www.yogiyo.co.kr/mobile/#/1050716/',
 'https://www.yogiyo.co.kr/mobile/#/240918/',
 'https://www.yogiyo.co.kr/mobile/#/1331457/',
 'https://www.yogiyo.co.kr/mobile/#/1280736/',
 'https://www.yogiyo.co.kr/mobile/#/1198375/',
 'https://www.yogiyo.co.kr/mobile/#/1302608/',
 'https://www.yogiyo.co.kr/mobile/#/285741/',
 'https://www.yogiyo.co.kr/mobile/#/259691/',
 'https://www.yogiyo.co.kr/mobile/#/19075/',
 'https://www.yogiyo.co.kr/mobile/#/1327746/',
 'https://www.yogiyo.co.kr/mobile/#/1230121/',
 'https://www.yogiyo.co.kr/mobile/#/486345/',
 'https://www.yogiyo.co.kr/mobile/#/356002/',
 'https://www.yogiyo.co.kr/mobile/#/1289094/',
 'https://www.yogiyo.co.kr/mobile/#/1209656/',
 'https://www.yogiyo.co.kr/mobile/#/1322394/',
 'https://www.yogiyo.co.kr/mobile/#/1241025/',
 'https://www.yogiyo.co.kr/mobile/#/1132842/',
 'https://www.yogiyo.co.kr/mobile/#/1227678/',
 'https://www.yogiyo.co.kr/mobile/#/191847/',
 'https://www.yogiyo.co.kr/mobile/#/553480/',
 'https://www.yogiyo.co.kr/mobile/#/1037198/',
 'https://www.yogiyo.co.kr/mobile/#/1190261/',
 'https://www.yogiyo.co.kr/mobile/#/1324192/',
 'https://www.yogiyo.co.kr/mobile/#/290200/',
 'https://www.yogiyo.co.kr/mobile/#/34846/',
 'https://www.yogiyo.co.kr/mobile/#/1209641/',
 'https://www.yogiyo.co.kr/mobile/#/1145550/',
 'https://www.yogiyo.co.kr/mobile/#/1181148/',
 'https://www.yogiyo.co.kr/mobile/#/1247239/',
 'https://www.yogiyo.co.kr/mobile/#/1029398/',
 'https://www.yogiyo.co.kr/mobile/#/269210/',
 'https://www.yogiyo.co.kr/mobile/#/464958/',
 'https://www.yogiyo.co.kr/mobile/#/1217015/',
 'https://www.yogiyo.co.kr/mobile/#/475341/',
 'https://www.yogiyo.co.kr/mobile/#/341436/',
 'https://www.yogiyo.co.kr/mobile/#/1241633/',
 'https://www.yogiyo.co.kr/mobile/#/1319832/',
 'https://www.yogiyo.co.kr/mobile/#/380854/',
 'https://www.yogiyo.co.kr/mobile/#/312507/',
 'https://www.yogiyo.co.kr/mobile/#/1237711/',
 'https://www.yogiyo.co.kr/mobile/#/1168383/',
 'https://www.yogiyo.co.kr/mobile/#/1040044/',
 'https://www.yogiyo.co.kr/mobile/#/297051/',
 'https://www.yogiyo.co.kr/mobile/#/1327902/',
 'https://www.yogiyo.co.kr/mobile/#/1195409/',
 'https://www.yogiyo.co.kr/mobile/#/230761/',
 'https://www.yogiyo.co.kr/mobile/#/1077895/',
 'https://www.yogiyo.co.kr/mobile/#/246766/',
 'https://www.yogiyo.co.kr/mobile/#/1228978/',
 'https://www.yogiyo.co.kr/mobile/#/1194625/',
 'https://www.yogiyo.co.kr/mobile/#/501121/',
 'https://www.yogiyo.co.kr/mobile/#/1309900/',
 'https://www.yogiyo.co.kr/mobile/#/313089/',
 'https://www.yogiyo.co.kr/mobile/#/1257356/',
 'https://www.yogiyo.co.kr/mobile/#/256840/',
 'https://www.yogiyo.co.kr/mobile/#/1331363/',
 'https://www.yogiyo.co.kr/mobile/#/1225950/',
 'https://www.yogiyo.co.kr/mobile/#/1052001/',
 'https://www.yogiyo.co.kr/mobile/#/373340/',
 'https://www.yogiyo.co.kr/mobile/#/294056/',
 'https://www.yogiyo.co.kr/mobile/#/246770/',
 'https://www.yogiyo.co.kr/mobile/#/1081040/',
 'https://www.yogiyo.co.kr/mobile/#/1333823/',
 'https://www.yogiyo.co.kr/mobile/#/1188066/',
 'https://www.yogiyo.co.kr/mobile/#/1234312/',
 'https://www.yogiyo.co.kr/mobile/#/1200517/',
 'https://www.yogiyo.co.kr/mobile/#/230554/',
 'https://www.yogiyo.co.kr/mobile/#/1201955/',
 'https://www.yogiyo.co.kr/mobile/#/1167365/',
 'https://www.yogiyo.co.kr/mobile/#/475597/',
 'https://www.yogiyo.co.kr/mobile/#/355768/',
 'https://www.yogiyo.co.kr/mobile/#/1130455/',
 'https://www.yogiyo.co.kr/mobile/#/303040/',
 'https://www.yogiyo.co.kr/mobile/#/467116/',
 'https://www.yogiyo.co.kr/mobile/#/471246/',
 'https://www.yogiyo.co.kr/mobile/#/82667/',
 'https://www.yogiyo.co.kr/mobile/#/424203/',
 'https://www.yogiyo.co.kr/mobile/#/1015567/',
 'https://www.yogiyo.co.kr/mobile/#/1057277/',
 'https://www.yogiyo.co.kr/mobile/#/246263/',
 'https://www.yogiyo.co.kr/mobile/#/1322208/',
 'https://www.yogiyo.co.kr/mobile/#/1277413/',
 'https://www.yogiyo.co.kr/mobile/#/28737/',
 'https://www.yogiyo.co.kr/mobile/#/1210097/',
 'https://www.yogiyo.co.kr/mobile/#/1254648/',
 'https://www.yogiyo.co.kr/mobile/#/236616/',
 'https://www.yogiyo.co.kr/mobile/#/449236/',
 'https://www.yogiyo.co.kr/mobile/#/1195546/',
 'https://www.yogiyo.co.kr/mobile/#/1274019/',
 'https://www.yogiyo.co.kr/mobile/#/1219170/',
 'https://www.yogiyo.co.kr/mobile/#/1315949/',
 'https://www.yogiyo.co.kr/mobile/#/243407/',
 'https://www.yogiyo.co.kr/mobile/#/1023973/',
 'https://www.yogiyo.co.kr/mobile/#/265554/',
 'https://www.yogiyo.co.kr/mobile/#/1325845/',
 'https://www.yogiyo.co.kr/mobile/#/542825/',
 'https://www.yogiyo.co.kr/mobile/#/470648/',
 'https://www.yogiyo.co.kr/mobile/#/1225984/',
 'https://www.yogiyo.co.kr/mobile/#/374030/',
 'https://www.yogiyo.co.kr/mobile/#/349035/',
 'https://www.yogiyo.co.kr/mobile/#/288936/',
 'https://www.yogiyo.co.kr/mobile/#/1087732/',
 'https://www.yogiyo.co.kr/mobile/#/1313946/',
 'https://www.yogiyo.co.kr/mobile/#/1147868/',
 'https://www.yogiyo.co.kr/mobile/#/1261815/',
 'https://www.yogiyo.co.kr/mobile/#/1295496/',
 'https://www.yogiyo.co.kr/mobile/#/431914/',
 'https://www.yogiyo.co.kr/mobile/#/1057955/',
 'https://www.yogiyo.co.kr/mobile/#/1326754/',
 'https://www.yogiyo.co.kr/mobile/#/1283642/',
 'https://www.yogiyo.co.kr/mobile/#/1295961/',
 'https://www.yogiyo.co.kr/mobile/#/1201312/',
 'https://www.yogiyo.co.kr/mobile/#/1254096/',
 'https://www.yogiyo.co.kr/mobile/#/1018537/',
 'https://www.yogiyo.co.kr/mobile/#/1146466/',
 'https://www.yogiyo.co.kr/mobile/#/1072167/',
 'https://www.yogiyo.co.kr/mobile/#/1059825/',
 'https://www.yogiyo.co.kr/mobile/#/486173/',
 'https://www.yogiyo.co.kr/mobile/#/1284774/',
 'https://www.yogiyo.co.kr/mobile/#/1188897/',
 'https://www.yogiyo.co.kr/mobile/#/1045197/',
 'https://www.yogiyo.co.kr/mobile/#/1297508/',
 'https://www.yogiyo.co.kr/mobile/#/1132166/',
 'https://www.yogiyo.co.kr/mobile/#/1023683/',
 'https://www.yogiyo.co.kr/mobile/#/569871/',
 'https://www.yogiyo.co.kr/mobile/#/1285977/',
 'https://www.yogiyo.co.kr/mobile/#/1146401/',
 'https://www.yogiyo.co.kr/mobile/#/545799/',
 'https://www.yogiyo.co.kr/mobile/#/1323149/',
 'https://www.yogiyo.co.kr/mobile/#/1009972/',
 'https://www.yogiyo.co.kr/mobile/#/1187228/',
 'https://www.yogiyo.co.kr/mobile/#/4577/',
 'https://www.yogiyo.co.kr/mobile/#/509088/',
 'https://www.yogiyo.co.kr/mobile/#/454132/',
 'https://www.yogiyo.co.kr/mobile/#/44833/',
 'https://www.yogiyo.co.kr/mobile/#/1238734/',
 'https://www.yogiyo.co.kr/mobile/#/1223608/',
 'https://www.yogiyo.co.kr/mobile/#/1129107/',
 'https://www.yogiyo.co.kr/mobile/#/327861/',
 'https://www.yogiyo.co.kr/mobile/#/1188107/',
 'https://www.yogiyo.co.kr/mobile/#/1299067/',
 'https://www.yogiyo.co.kr/mobile/#/1254392/',
 'https://www.yogiyo.co.kr/mobile/#/1258079/',
 'https://www.yogiyo.co.kr/mobile/#/1210964/',
 'https://www.yogiyo.co.kr/mobile/#/1018954/',
 'https://www.yogiyo.co.kr/mobile/#/468086/',
 'https://www.yogiyo.co.kr/mobile/#/453224/',
 'https://www.yogiyo.co.kr/mobile/#/1081477/',
 'https://www.yogiyo.co.kr/mobile/#/1305443/',
 'https://www.yogiyo.co.kr/mobile/#/568974/',
 'https://www.yogiyo.co.kr/mobile/#/1145391/',
 'https://www.yogiyo.co.kr/mobile/#/1269722/',
 'https://www.yogiyo.co.kr/mobile/#/1228756/',
 'https://www.yogiyo.co.kr/mobile/#/1260805/',
 'https://www.yogiyo.co.kr/mobile/#/1128028/',
 'https://www.yogiyo.co.kr/mobile/#/399961/',
 'https://www.yogiyo.co.kr/mobile/#/1286144/',
 'https://www.yogiyo.co.kr/mobile/#/402559/',
 'https://www.yogiyo.co.kr/mobile/#/415857/',
 'https://www.yogiyo.co.kr/mobile/#/1237637/',
 'https://www.yogiyo.co.kr/mobile/#/1257490/',
 'https://www.yogiyo.co.kr/mobile/#/374892/',
 'https://www.yogiyo.co.kr/mobile/#/307590/',
 'https://www.yogiyo.co.kr/mobile/#/386551/',
 'https://www.yogiyo.co.kr/mobile/#/1143495/',
 'https://www.yogiyo.co.kr/mobile/#/1208751/',
 'https://www.yogiyo.co.kr/mobile/#/295184/',
 'https://www.yogiyo.co.kr/mobile/#/1176883/',
 'https://www.yogiyo.co.kr/mobile/#/259071/',
 'https://www.yogiyo.co.kr/mobile/#/571233/',
 'https://www.yogiyo.co.kr/mobile/#/1257031/',
 'https://www.yogiyo.co.kr/mobile/#/34750/',
 'https://www.yogiyo.co.kr/mobile/#/342354/',
 'https://www.yogiyo.co.kr/mobile/#/1144758/',
 'https://www.yogiyo.co.kr/mobile/#/576741/',
 'https://www.yogiyo.co.kr/mobile/#/587882/',
 'https://www.yogiyo.co.kr/mobile/#/1317175/',
 'https://www.yogiyo.co.kr/mobile/#/1015125/',
 'https://www.yogiyo.co.kr/mobile/#/297232/',
 'https://www.yogiyo.co.kr/mobile/#/370517/',
 'https://www.yogiyo.co.kr/mobile/#/557981/',
 'https://www.yogiyo.co.kr/mobile/#/314803/',
 'https://www.yogiyo.co.kr/mobile/#/1142076/',
 'https://www.yogiyo.co.kr/mobile/#/257776/',
 'https://www.yogiyo.co.kr/mobile/#/378331/',
 'https://www.yogiyo.co.kr/mobile/#/1036412/',
 'https://www.yogiyo.co.kr/mobile/#/553742/',
 'https://www.yogiyo.co.kr/mobile/#/1293224/',
 'https://www.yogiyo.co.kr/mobile/#/43252/',
 'https://www.yogiyo.co.kr/mobile/#/1152090/',
 'https://www.yogiyo.co.kr/mobile/#/245488/',
 'https://www.yogiyo.co.kr/mobile/#/294300/',
 'https://www.yogiyo.co.kr/mobile/#/536330/',
 'https://www.yogiyo.co.kr/mobile/#/1290040/',
 'https://www.yogiyo.co.kr/mobile/#/469330/',
 'https://www.yogiyo.co.kr/mobile/#/1179335/',
 'https://www.yogiyo.co.kr/mobile/#/1199850/',
 'https://www.yogiyo.co.kr/mobile/#/1176854/',
 'https://www.yogiyo.co.kr/mobile/#/352316/',
 'https://www.yogiyo.co.kr/mobile/#/1110077/',
 'https://www.yogiyo.co.kr/mobile/#/1238764/',
 'https://www.yogiyo.co.kr/mobile/#/257483/',
 'https://www.yogiyo.co.kr/mobile/#/1232231/',
 'https://www.yogiyo.co.kr/mobile/#/1311022/',
 'https://www.yogiyo.co.kr/mobile/#/1188805/',
 'https://www.yogiyo.co.kr/mobile/#/1316542/',
 'https://www.yogiyo.co.kr/mobile/#/305447/',
 'https://www.yogiyo.co.kr/mobile/#/342801/',
 'https://www.yogiyo.co.kr/mobile/#/499891/',
 'https://www.yogiyo.co.kr/mobile/#/547736/',
 'https://www.yogiyo.co.kr/mobile/#/357765/',
 'https://www.yogiyo.co.kr/mobile/#/1212297/',
 'https://www.yogiyo.co.kr/mobile/#/236615/',
 'https://www.yogiyo.co.kr/mobile/#/464691/',
 'https://www.yogiyo.co.kr/mobile/#/348767/',
 'https://www.yogiyo.co.kr/mobile/#/1301122/',
 'https://www.yogiyo.co.kr/mobile/#/1295255/',
 'https://www.yogiyo.co.kr/mobile/#/1169238/',
 'https://www.yogiyo.co.kr/mobile/#/1029648/',
 'https://www.yogiyo.co.kr/mobile/#/413479/',
 'https://www.yogiyo.co.kr/mobile/#/576896/',
 'https://www.yogiyo.co.kr/mobile/#/1248997/',
 'https://www.yogiyo.co.kr/mobile/#/1112169/',
 'https://www.yogiyo.co.kr/mobile/#/444132/',
 'https://www.yogiyo.co.kr/mobile/#/1054045/',
 'https://www.yogiyo.co.kr/mobile/#/1033685/',
 'https://www.yogiyo.co.kr/mobile/#/242345/',
 'https://www.yogiyo.co.kr/mobile/#/91984/',
 'https://www.yogiyo.co.kr/mobile/#/426762/',
 'https://www.yogiyo.co.kr/mobile/#/1087698/',
 'https://www.yogiyo.co.kr/mobile/#/354699/',
 'https://www.yogiyo.co.kr/mobile/#/1168201/',
 'https://www.yogiyo.co.kr/mobile/#/1297546/',
 'https://www.yogiyo.co.kr/mobile/#/1326751/',
 'https://www.yogiyo.co.kr/mobile/#/435201/',
 'https://www.yogiyo.co.kr/mobile/#/1251225/',
 'https://www.yogiyo.co.kr/mobile/#/399458/',
 'https://www.yogiyo.co.kr/mobile/#/531080/',
 'https://www.yogiyo.co.kr/mobile/#/326371/',
 'https://www.yogiyo.co.kr/mobile/#/1323494/',
 'https://www.yogiyo.co.kr/mobile/#/1262583/',
 'https://www.yogiyo.co.kr/mobile/#/1281189/',
 'https://www.yogiyo.co.kr/mobile/#/1273780/',
 'https://www.yogiyo.co.kr/mobile/#/490502/',
 'https://www.yogiyo.co.kr/mobile/#/82030/',
 'https://www.yogiyo.co.kr/mobile/#/1012285/',
 'https://www.yogiyo.co.kr/mobile/#/1301178/',
 'https://www.yogiyo.co.kr/mobile/#/277803/',
 'https://www.yogiyo.co.kr/mobile/#/1039771/',
 'https://www.yogiyo.co.kr/mobile/#/1229870/',
 'https://www.yogiyo.co.kr/mobile/#/335533/',
 'https://www.yogiyo.co.kr/mobile/#/249201/',
 'https://www.yogiyo.co.kr/mobile/#/427615/',
 'https://www.yogiyo.co.kr/mobile/#/1083650/',
 'https://www.yogiyo.co.kr/mobile/#/1090048/',
 'https://www.yogiyo.co.kr/mobile/#/1227387/',
 'https://www.yogiyo.co.kr/mobile/#/1127009/',
 'https://www.yogiyo.co.kr/mobile/#/1088804/',
 'https://www.yogiyo.co.kr/mobile/#/236458/',
 'https://www.yogiyo.co.kr/mobile/#/1226657/',
 'https://www.yogiyo.co.kr/mobile/#/440654/',
 'https://www.yogiyo.co.kr/mobile/#/369210/',
 'https://www.yogiyo.co.kr/mobile/#/1297519/',
 'https://www.yogiyo.co.kr/mobile/#/363643/',
 'https://www.yogiyo.co.kr/mobile/#/551973/']

In [229]:
failed_urls = failed_urls[1:]
failed_urls

['https://www.yogiyo.co.kr/mobile/#/472721/',
 'https://www.yogiyo.co.kr/mobile/#/1096085/',
 'https://www.yogiyo.co.kr/mobile/#/1085741/',
 'https://www.yogiyo.co.kr/mobile/#/1176856/',
 'https://www.yogiyo.co.kr/mobile/#/1184934/',
 'https://www.yogiyo.co.kr/mobile/#/319827/',
 'https://www.yogiyo.co.kr/mobile/#/1323653/',
 'https://www.yogiyo.co.kr/mobile/#/520369/',
 'https://www.yogiyo.co.kr/mobile/#/443038/',
 'https://www.yogiyo.co.kr/mobile/#/83302/',
 'https://www.yogiyo.co.kr/mobile/#/586014/',
 'https://www.yogiyo.co.kr/mobile/#/425565/',
 'https://www.yogiyo.co.kr/mobile/#/1061927/',
 'https://www.yogiyo.co.kr/mobile/#/24498/',
 'https://www.yogiyo.co.kr/mobile/#/468676/',
 'https://www.yogiyo.co.kr/mobile/#/1323654/',
 'https://www.yogiyo.co.kr/mobile/#/40436/',
 'https://www.yogiyo.co.kr/mobile/#/317294/',
 'https://www.yogiyo.co.kr/mobile/#/322041/',
 'https://www.yogiyo.co.kr/mobile/#/369213/',
 'https://www.yogiyo.co.kr/mobile/#/1189693/',
 'https://www.yogiyo.co.kr/mo

In [230]:
main = pd.read_csv("dataset/review_data_0406.csv", index_col=0)
main[main['store']=='맘스터치피자앤치킨-건대점'] 

,store,address,taste_score,quantity_score,delivery_score,review,review_time


In [231]:
review_data = []

soup = BeautifulSoup(driver.page_source, 'lxml')

# 가게 기본 데이터
store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

# 리뷰 데이터
review_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
review_btn.click()
time.sleep(1)

def collect_reviews():
    soup = BeautifulSoup(driver.page_source, 'lxml')
    review_elements = soup.find_all('li', class_="list-group-item star-point ng-scope")
    for review_element in review_elements:
        score = review_element.find('span', class_='category')
        categorical_score = score.find_all('span', class_='points ng-binding')
        taste_score = categorical_score[0].text.strip() if len(categorical_score) > 0 else None
        quantity_score = categorical_score[1].text.strip() if len(categorical_score) > 1 else None
        review_time = review_element.find('span', class_='review-time ng-binding').text.strip()
        delivery_score = categorical_score[2].text.strip() if len(categorical_score) > 2 else None
        review_content = review_element.find('p', class_='ng-binding').text.strip()

        review_data.append({
            'store': store_name,
            'address': address,
            'taste_score': taste_score,
            'quantity_score': quantity_score,
            'delivery_score': delivery_score,
            'review': review_content,
            'review_time': review_time
        })

# 더보기 버튼 클릭 함수
def click_more_button():
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        plus_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, plus_btn_xpath))
        )
        plus_btn.click()
        return True
    except:
        return False

i = 0
while True:
    i += 1
    var = i * 10 + 2
    plus_btn_xpath = f'/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[{var}]/a'

    if not click_more_button():
        break

    if i % 50 == 0:
        print(f"{i*10}번째 리뷰 여는 중...")

    time.sleep(1.5)  # 클릭 후 대기 시간 추가

collect_reviews()

review_df = pd.DataFrame(review_data).drop_duplicates().reset_index(drop=True)


500번째 리뷰 여는 중...
1000번째 리뷰 여는 중...


In [206]:
i = 0
while True:
    i += 1
    var = i * 10 + 3872
    plus_btn_xpath = f'/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[{var}]/a'

    if not click_more_button():
        break

    if i % 50 == 0:
        print(f"{i*10}번째 리뷰 여는 중...")

    time.sleep(0.7)  # 클릭 후 대기 시간 추가

# //*[@id="review"]/li[152]
#  //*[@id="review"]/li[222]
# //*[@id="review"]/li[1162]
# //*[@id="review"]/li[1982]
# //*[@id="review"]/li[3882]
#

500번째 리뷰 여는 중...
1000번째 리뷰 여는 중...


In [194]:
collect_reviews()
review_df = pd.DataFrame(review_data).drop_duplicates().reset_index(drop=True)

In [232]:
review_df

,store,address,taste_score,quantity_score,delivery_score,review,review_time
0,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,None,양이 엄청 많습니다\n배달 빠릅니다\n맛있습니다\n따뜻합니다\n다시 시킬것 같아요!,2시간 전
1,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,None,맛있게 먹었습니다..,4시간 전
2,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,None,감튀맛있네요 치킨도 맛있고,어제
3,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,None,와 맘터 순살 맛있네요 양도 많고 굿,어제
4,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,5,맛있게 잘먹었습니다,일주일 전
...,...,...,...,...,...,...,...
1113,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,5,구성도 풍성하고 배달도 빨리 왔습니다\n싸이 순살이 리뉴얼이 됐는지 훨씬 양도 많고...,2022년 5월 21일
1114,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,5,양많고 맛괜찮고 배달빠름,2022년 5월 19일
1115,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,5,처음 시켜보는곳이였는데 생각이상으로 양도 많고 맛있었습니다 다음에도 또 시킬거 같아...,2022년 5월 19일
1116,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,5,"너무 맛있어서 먹다가 찍었어요,, 어떡하징",2022년 5월 17일


In [233]:
main = pd.concat([main, review_df], axis=0).reset_index(drop=True)
main.to_csv('dataset/review_data_0406.csv')
main

,store,address,taste_score,quantity_score,delivery_score,review,review_time
0,24시채린,서울특별시 광진구 중곡동 140-3 1층,5.0,5.0,5.0,너무 맛있게 잘 먹었습니다ㅎㅎ 다음에도 여기서 시켜먹을게요!,9시간 전
1,24시채린,서울특별시 광진구 중곡동 140-3 1층,5.0,5.0,NaN,잡채밥 너무 맛있어요,11시간 전
2,24시채린,서울특별시 광진구 중곡동 140-3 1층,5.0,5.0,NaN,맛있게 잘 먹었습니다,어제
3,24시채린,서울특별시 광진구 중곡동 140-3 1층,5.0,5.0,NaN,맛과 양이 타중국집과 비교할수 없네요. 채린 음식 먹다보면 배터질거 같아요. ^^,2일 전
4,24시채린,서울특별시 광진구 중곡동 140-3 1층,4.0,4.0,NaN,잘 먹 었 습 니 다,2일 전
...,...,...,...,...,...,...,...
213067,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,5,구성도 풍성하고 배달도 빨리 왔습니다\n싸이 순살이 리뉴얼이 됐는지 훨씬 양도 많고...,2022년 5월 21일
213068,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,5,양많고 맛괜찮고 배달빠름,2022년 5월 19일
213069,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,5,처음 시켜보는곳이였는데 생각이상으로 양도 많고 맛있었습니다 다음에도 또 시킬거 같아...,2022년 5월 19일
213070,맘스터치피자앤치킨-건대점,서울특별시 광진구 화양동 12-34 1층,5,5,5,"너무 맛있어서 먹다가 찍었어요,, 어떡하징",2022년 5월 17일


### **남은 가게 크롤링** ###

In [234]:
remained_list = ['https://www.yogiyo.co.kr/mobile/#/405486/',
 'https://www.yogiyo.co.kr/mobile/#/1324325/',
 'https://www.yogiyo.co.kr/mobile/#/549089/',
 'https://www.yogiyo.co.kr/mobile/#/1085741/',
 'https://www.yogiyo.co.kr/mobile/#/1176856/',
 'https://www.yogiyo.co.kr/mobile/#/1184934/',
 'https://www.yogiyo.co.kr/mobile/#/319827/',
 'https://www.yogiyo.co.kr/mobile/#/1323653/',
 'https://www.yogiyo.co.kr/mobile/#/520369/',
 'https://www.yogiyo.co.kr/mobile/#/443038/',
 'https://www.yogiyo.co.kr/mobile/#/83302/',
 'https://www.yogiyo.co.kr/mobile/#/586014/',
 'https://www.yogiyo.co.kr/mobile/#/425565/',
 'https://www.yogiyo.co.kr/mobile/#/1061927/',
 'https://www.yogiyo.co.kr/mobile/#/24498/',
 'https://www.yogiyo.co.kr/mobile/#/468676/',
 'https://www.yogiyo.co.kr/mobile/#/1323654/',
 'https://www.yogiyo.co.kr/mobile/#/40436/',
 'https://www.yogiyo.co.kr/mobile/#/317294/',
 'https://www.yogiyo.co.kr/mobile/#/322041/',
 'https://www.yogiyo.co.kr/mobile/#/369213/',
 'https://www.yogiyo.co.kr/mobile/#/1189693/',
 'https://www.yogiyo.co.kr/mobile/#/1287950/',
 'https://www.yogiyo.co.kr/mobile/#/1228680/',
 'https://www.yogiyo.co.kr/mobile/#/4597/',
 'https://www.yogiyo.co.kr/mobile/#/1270336/',
 'https://www.yogiyo.co.kr/mobile/#/263183/',
 'https://www.yogiyo.co.kr/mobile/#/474247/',
 'https://www.yogiyo.co.kr/mobile/#/1103570/',
 'https://www.yogiyo.co.kr/mobile/#/249743/',
 'https://www.yogiyo.co.kr/mobile/#/1213364/',
 'https://www.yogiyo.co.kr/mobile/#/580446/',
 'https://www.yogiyo.co.kr/mobile/#/276408/',
 'https://www.yogiyo.co.kr/mobile/#/1081834/',
 'https://www.yogiyo.co.kr/mobile/#/527346/',
 'https://www.yogiyo.co.kr/mobile/#/319449/',
 'https://www.yogiyo.co.kr/mobile/#/1298716/',
 'https://www.yogiyo.co.kr/mobile/#/1295091/',
 'https://www.yogiyo.co.kr/mobile/#/506487/',
 'https://www.yogiyo.co.kr/mobile/#/1137636/',
 'https://www.yogiyo.co.kr/mobile/#/415834/',
 'https://www.yogiyo.co.kr/mobile/#/1321949/',
 'https://www.yogiyo.co.kr/mobile/#/1031136/',
 'https://www.yogiyo.co.kr/mobile/#/370351/',
 'https://www.yogiyo.co.kr/mobile/#/1020358/',
 'https://www.yogiyo.co.kr/mobile/#/1286682/',
 'https://www.yogiyo.co.kr/mobile/#/1217367/',
 'https://www.yogiyo.co.kr/mobile/#/1180412/',
 'https://www.yogiyo.co.kr/mobile/#/1333818/',
 'https://www.yogiyo.co.kr/mobile/#/1171243/',
 'https://www.yogiyo.co.kr/mobile/#/1313934/',
 'https://www.yogiyo.co.kr/mobile/#/231147/',
 'https://www.yogiyo.co.kr/mobile/#/386235/',
 'https://www.yogiyo.co.kr/mobile/#/454778/',
 'https://www.yogiyo.co.kr/mobile/#/251628/',
 'https://www.yogiyo.co.kr/mobile/#/1271730/',
 'https://www.yogiyo.co.kr/mobile/#/520695/',
 'https://www.yogiyo.co.kr/mobile/#/1313347/',
 'https://www.yogiyo.co.kr/mobile/#/1247188/',
 'https://www.yogiyo.co.kr/mobile/#/1175900/',
 'https://www.yogiyo.co.kr/mobile/#/1029710/',
 'https://www.yogiyo.co.kr/mobile/#/1050716/',
 'https://www.yogiyo.co.kr/mobile/#/240918/',
 'https://www.yogiyo.co.kr/mobile/#/1331457/',
 'https://www.yogiyo.co.kr/mobile/#/1280736/',
 'https://www.yogiyo.co.kr/mobile/#/1198375/',
 'https://www.yogiyo.co.kr/mobile/#/1302608/',
 'https://www.yogiyo.co.kr/mobile/#/285741/',
 'https://www.yogiyo.co.kr/mobile/#/259691/',
 'https://www.yogiyo.co.kr/mobile/#/19075/',
 'https://www.yogiyo.co.kr/mobile/#/1327746/',
 'https://www.yogiyo.co.kr/mobile/#/486345/',
 'https://www.yogiyo.co.kr/mobile/#/356002/',
 'https://www.yogiyo.co.kr/mobile/#/1289094/',
 'https://www.yogiyo.co.kr/mobile/#/1209656/',
 'https://www.yogiyo.co.kr/mobile/#/1322394/',
 'https://www.yogiyo.co.kr/mobile/#/1132842/',
 'https://www.yogiyo.co.kr/mobile/#/1227678/',
 'https://www.yogiyo.co.kr/mobile/#/191847/',
 'https://www.yogiyo.co.kr/mobile/#/553480/',
 'https://www.yogiyo.co.kr/mobile/#/1037198/',
 'https://www.yogiyo.co.kr/mobile/#/1190261/',
 'https://www.yogiyo.co.kr/mobile/#/1324192/',
 'https://www.yogiyo.co.kr/mobile/#/290200/',
 'https://www.yogiyo.co.kr/mobile/#/34846/',
 'https://www.yogiyo.co.kr/mobile/#/1209641/',
 'https://www.yogiyo.co.kr/mobile/#/1145550/',
 'https://www.yogiyo.co.kr/mobile/#/1181148/',
 'https://www.yogiyo.co.kr/mobile/#/1247239/',
 'https://www.yogiyo.co.kr/mobile/#/1029398/',
 'https://www.yogiyo.co.kr/mobile/#/269210/',
 'https://www.yogiyo.co.kr/mobile/#/464958/',
 'https://www.yogiyo.co.kr/mobile/#/1217015/',
 'https://www.yogiyo.co.kr/mobile/#/475341/',
 'https://www.yogiyo.co.kr/mobile/#/341436/',
 'https://www.yogiyo.co.kr/mobile/#/1241633/',
 'https://www.yogiyo.co.kr/mobile/#/1319832/',
 'https://www.yogiyo.co.kr/mobile/#/380854/',
 'https://www.yogiyo.co.kr/mobile/#/312507/',
 'https://www.yogiyo.co.kr/mobile/#/1237711/',
 'https://www.yogiyo.co.kr/mobile/#/1168383/',
 'https://www.yogiyo.co.kr/mobile/#/1040044/',
 'https://www.yogiyo.co.kr/mobile/#/297051/',
 'https://www.yogiyo.co.kr/mobile/#/1327902/',
 'https://www.yogiyo.co.kr/mobile/#/1195409/',
 'https://www.yogiyo.co.kr/mobile/#/1077895/',
 'https://www.yogiyo.co.kr/mobile/#/246766/',
 'https://www.yogiyo.co.kr/mobile/#/1228978/',
 'https://www.yogiyo.co.kr/mobile/#/1194625/',
 'https://www.yogiyo.co.kr/mobile/#/501121/',
 'https://www.yogiyo.co.kr/mobile/#/1309900/',
 'https://www.yogiyo.co.kr/mobile/#/313089/',
 'https://www.yogiyo.co.kr/mobile/#/1257356/',
 'https://www.yogiyo.co.kr/mobile/#/256840/',
 'https://www.yogiyo.co.kr/mobile/#/1331363/',
 'https://www.yogiyo.co.kr/mobile/#/1225950/',
 'https://www.yogiyo.co.kr/mobile/#/1052001/',
 'https://www.yogiyo.co.kr/mobile/#/373340/',
 'https://www.yogiyo.co.kr/mobile/#/294056/',
 'https://www.yogiyo.co.kr/mobile/#/246770/',
 'https://www.yogiyo.co.kr/mobile/#/1081040/',
 'https://www.yogiyo.co.kr/mobile/#/1333823/',
 'https://www.yogiyo.co.kr/mobile/#/1188066/',
 'https://www.yogiyo.co.kr/mobile/#/1234312/',
 'https://www.yogiyo.co.kr/mobile/#/1200517/',
 'https://www.yogiyo.co.kr/mobile/#/230554/',
 'https://www.yogiyo.co.kr/mobile/#/1201955/',
 'https://www.yogiyo.co.kr/mobile/#/1167365/',
 'https://www.yogiyo.co.kr/mobile/#/475597/',
 'https://www.yogiyo.co.kr/mobile/#/355768/',
 'https://www.yogiyo.co.kr/mobile/#/1130455/',
 'https://www.yogiyo.co.kr/mobile/#/303040/',
 'https://www.yogiyo.co.kr/mobile/#/467116/',
 'https://www.yogiyo.co.kr/mobile/#/471246/',
 'https://www.yogiyo.co.kr/mobile/#/424203/',
 'https://www.yogiyo.co.kr/mobile/#/1015567/',
 'https://www.yogiyo.co.kr/mobile/#/1057277/',
 'https://www.yogiyo.co.kr/mobile/#/246263/',
 'https://www.yogiyo.co.kr/mobile/#/1322208/',
 'https://www.yogiyo.co.kr/mobile/#/1277413/',
 'https://www.yogiyo.co.kr/mobile/#/28737/',
 'https://www.yogiyo.co.kr/mobile/#/1210097/',
 'https://www.yogiyo.co.kr/mobile/#/1254648/',
 'https://www.yogiyo.co.kr/mobile/#/236616/',
 'https://www.yogiyo.co.kr/mobile/#/449236/',
 'https://www.yogiyo.co.kr/mobile/#/1195546/',
 'https://www.yogiyo.co.kr/mobile/#/1274019/',
 'https://www.yogiyo.co.kr/mobile/#/1219170/',
 'https://www.yogiyo.co.kr/mobile/#/1315949/',
 'https://www.yogiyo.co.kr/mobile/#/243407/',
 'https://www.yogiyo.co.kr/mobile/#/1023973/',
 'https://www.yogiyo.co.kr/mobile/#/265554/',
 'https://www.yogiyo.co.kr/mobile/#/1325845/',
 'https://www.yogiyo.co.kr/mobile/#/542825/',
 'https://www.yogiyo.co.kr/mobile/#/470648/',
 'https://www.yogiyo.co.kr/mobile/#/1225984/',
 'https://www.yogiyo.co.kr/mobile/#/374030/',
 'https://www.yogiyo.co.kr/mobile/#/349035/',
 'https://www.yogiyo.co.kr/mobile/#/288936/',
 'https://www.yogiyo.co.kr/mobile/#/1087732/',
 'https://www.yogiyo.co.kr/mobile/#/1313946/',
 'https://www.yogiyo.co.kr/mobile/#/1147868/',
 'https://www.yogiyo.co.kr/mobile/#/1261815/',
 'https://www.yogiyo.co.kr/mobile/#/1295496/',
 'https://www.yogiyo.co.kr/mobile/#/431914/',
 'https://www.yogiyo.co.kr/mobile/#/1057955/',
 'https://www.yogiyo.co.kr/mobile/#/1326754/',
 'https://www.yogiyo.co.kr/mobile/#/1283642/',
 'https://www.yogiyo.co.kr/mobile/#/1295961/',
 'https://www.yogiyo.co.kr/mobile/#/1201312/',
 'https://www.yogiyo.co.kr/mobile/#/1254096/',
 'https://www.yogiyo.co.kr/mobile/#/1018537/',
 'https://www.yogiyo.co.kr/mobile/#/1146466/',
 'https://www.yogiyo.co.kr/mobile/#/1072167/',
 'https://www.yogiyo.co.kr/mobile/#/1059825/',
 'https://www.yogiyo.co.kr/mobile/#/486173/',
 'https://www.yogiyo.co.kr/mobile/#/1284774/',
 'https://www.yogiyo.co.kr/mobile/#/1188897/',
 'https://www.yogiyo.co.kr/mobile/#/1045197/',
 'https://www.yogiyo.co.kr/mobile/#/1297508/',
 'https://www.yogiyo.co.kr/mobile/#/1132166/',
 'https://www.yogiyo.co.kr/mobile/#/1023683/',
 'https://www.yogiyo.co.kr/mobile/#/569871/',
 'https://www.yogiyo.co.kr/mobile/#/1285977/',
 'https://www.yogiyo.co.kr/mobile/#/1146401/',
 'https://www.yogiyo.co.kr/mobile/#/545799/',
 'https://www.yogiyo.co.kr/mobile/#/1323149/',
 'https://www.yogiyo.co.kr/mobile/#/1009972/',
 'https://www.yogiyo.co.kr/mobile/#/1187228/',
 'https://www.yogiyo.co.kr/mobile/#/4577/',
 'https://www.yogiyo.co.kr/mobile/#/509088/',
 'https://www.yogiyo.co.kr/mobile/#/454132/',
 'https://www.yogiyo.co.kr/mobile/#/44833/',
 'https://www.yogiyo.co.kr/mobile/#/1238734/',
 'https://www.yogiyo.co.kr/mobile/#/1223608/',
 'https://www.yogiyo.co.kr/mobile/#/1129107/',
 'https://www.yogiyo.co.kr/mobile/#/327861/',
 'https://www.yogiyo.co.kr/mobile/#/1188107/',
 'https://www.yogiyo.co.kr/mobile/#/1299067/',
 'https://www.yogiyo.co.kr/mobile/#/1254392/',
 'https://www.yogiyo.co.kr/mobile/#/1258079/',
 'https://www.yogiyo.co.kr/mobile/#/1210964/',
 'https://www.yogiyo.co.kr/mobile/#/1018954/',
 'https://www.yogiyo.co.kr/mobile/#/468086/',
 'https://www.yogiyo.co.kr/mobile/#/453224/',
 'https://www.yogiyo.co.kr/mobile/#/1081477/',
 'https://www.yogiyo.co.kr/mobile/#/1305443/',
 'https://www.yogiyo.co.kr/mobile/#/568974/',
 'https://www.yogiyo.co.kr/mobile/#/1145391/',
 'https://www.yogiyo.co.kr/mobile/#/1269722/',
 'https://www.yogiyo.co.kr/mobile/#/1228756/',
 'https://www.yogiyo.co.kr/mobile/#/1260805/',
 'https://www.yogiyo.co.kr/mobile/#/1128028/',
 'https://www.yogiyo.co.kr/mobile/#/399961/',
 'https://www.yogiyo.co.kr/mobile/#/1286144/',
 'https://www.yogiyo.co.kr/mobile/#/402559/',
 'https://www.yogiyo.co.kr/mobile/#/415857/',
 'https://www.yogiyo.co.kr/mobile/#/1237637/',
 'https://www.yogiyo.co.kr/mobile/#/1257490/',
 'https://www.yogiyo.co.kr/mobile/#/374892/',
 'https://www.yogiyo.co.kr/mobile/#/307590/',
 'https://www.yogiyo.co.kr/mobile/#/386551/',
 'https://www.yogiyo.co.kr/mobile/#/1143495/',
 'https://www.yogiyo.co.kr/mobile/#/1208751/',
 'https://www.yogiyo.co.kr/mobile/#/295184/',
 'https://www.yogiyo.co.kr/mobile/#/1176883/',
 'https://www.yogiyo.co.kr/mobile/#/259071/',
 'https://www.yogiyo.co.kr/mobile/#/1327102/',
 'https://www.yogiyo.co.kr/mobile/#/571233/',
 'https://www.yogiyo.co.kr/mobile/#/1257031/',
 'https://www.yogiyo.co.kr/mobile/#/34750/',
 'https://www.yogiyo.co.kr/mobile/#/342354/',
 'https://www.yogiyo.co.kr/mobile/#/1144758/',
 'https://www.yogiyo.co.kr/mobile/#/576741/',
 'https://www.yogiyo.co.kr/mobile/#/587882/',
 'https://www.yogiyo.co.kr/mobile/#/1317175/',
 'https://www.yogiyo.co.kr/mobile/#/1015125/',
 'https://www.yogiyo.co.kr/mobile/#/297232/',
 'https://www.yogiyo.co.kr/mobile/#/370517/',
 'https://www.yogiyo.co.kr/mobile/#/557981/',
 'https://www.yogiyo.co.kr/mobile/#/314803/',
 'https://www.yogiyo.co.kr/mobile/#/1142076/',
 'https://www.yogiyo.co.kr/mobile/#/257776/',
 'https://www.yogiyo.co.kr/mobile/#/378331/',
 'https://www.yogiyo.co.kr/mobile/#/1036412/',
 'https://www.yogiyo.co.kr/mobile/#/553742/',
 'https://www.yogiyo.co.kr/mobile/#/1293224/',
 'https://www.yogiyo.co.kr/mobile/#/1152090/',
 'https://www.yogiyo.co.kr/mobile/#/294300/',
 'https://www.yogiyo.co.kr/mobile/#/536330/',
 'https://www.yogiyo.co.kr/mobile/#/1290040/',
 'https://www.yogiyo.co.kr/mobile/#/469330/',
 'https://www.yogiyo.co.kr/mobile/#/1179335/',
 'https://www.yogiyo.co.kr/mobile/#/1330112/',
 'https://www.yogiyo.co.kr/mobile/#/1199850/',
 'https://www.yogiyo.co.kr/mobile/#/1176854/',
 'https://www.yogiyo.co.kr/mobile/#/352316/',
 'https://www.yogiyo.co.kr/mobile/#/1110077/',
 'https://www.yogiyo.co.kr/mobile/#/1238764/',
 'https://www.yogiyo.co.kr/mobile/#/257483/',
 'https://www.yogiyo.co.kr/mobile/#/1232231/',
 'https://www.yogiyo.co.kr/mobile/#/1311022/',
 'https://www.yogiyo.co.kr/mobile/#/1188805/',
 'https://www.yogiyo.co.kr/mobile/#/1316542/',
 'https://www.yogiyo.co.kr/mobile/#/305447/',
 'https://www.yogiyo.co.kr/mobile/#/342801/',
 'https://www.yogiyo.co.kr/mobile/#/499891/',
 'https://www.yogiyo.co.kr/mobile/#/547736/',
 'https://www.yogiyo.co.kr/mobile/#/357765/',
 'https://www.yogiyo.co.kr/mobile/#/1212297/',
 'https://www.yogiyo.co.kr/mobile/#/236615/',
 'https://www.yogiyo.co.kr/mobile/#/464691/',
 'https://www.yogiyo.co.kr/mobile/#/348767/',
 'https://www.yogiyo.co.kr/mobile/#/1301122/',
 'https://www.yogiyo.co.kr/mobile/#/1295255/',
 'https://www.yogiyo.co.kr/mobile/#/1169238/',
 'https://www.yogiyo.co.kr/mobile/#/1029648/',
 'https://www.yogiyo.co.kr/mobile/#/413479/',
 'https://www.yogiyo.co.kr/mobile/#/576896/',
 'https://www.yogiyo.co.kr/mobile/#/1248997/',
 'https://www.yogiyo.co.kr/mobile/#/1112169/',
 'https://www.yogiyo.co.kr/mobile/#/444132/',
 'https://www.yogiyo.co.kr/mobile/#/1054045/',
 'https://www.yogiyo.co.kr/mobile/#/1033685/',
 'https://www.yogiyo.co.kr/mobile/#/242345/',
 'https://www.yogiyo.co.kr/mobile/#/91984/',
 'https://www.yogiyo.co.kr/mobile/#/426762/',
 'https://www.yogiyo.co.kr/mobile/#/1087698/',
 'https://www.yogiyo.co.kr/mobile/#/354699/',
 'https://www.yogiyo.co.kr/mobile/#/1168201/',
 'https://www.yogiyo.co.kr/mobile/#/1297546/',
 'https://www.yogiyo.co.kr/mobile/#/1326751/',
 'https://www.yogiyo.co.kr/mobile/#/435201/',
 'https://www.yogiyo.co.kr/mobile/#/1251225/',
 'https://www.yogiyo.co.kr/mobile/#/399458/',
 'https://www.yogiyo.co.kr/mobile/#/531080/',
 'https://www.yogiyo.co.kr/mobile/#/326371/',
 'https://www.yogiyo.co.kr/mobile/#/1323494/',
 'https://www.yogiyo.co.kr/mobile/#/1262583/',
 'https://www.yogiyo.co.kr/mobile/#/1281189/',
 'https://www.yogiyo.co.kr/mobile/#/1273780/',
 'https://www.yogiyo.co.kr/mobile/#/490502/',
 'https://www.yogiyo.co.kr/mobile/#/82030/',
 'https://www.yogiyo.co.kr/mobile/#/1012285/',
 'https://www.yogiyo.co.kr/mobile/#/1301178/',
 'https://www.yogiyo.co.kr/mobile/#/277803/',
 'https://www.yogiyo.co.kr/mobile/#/1039771/',
 'https://www.yogiyo.co.kr/mobile/#/1229870/',
 'https://www.yogiyo.co.kr/mobile/#/335533/',
 'https://www.yogiyo.co.kr/mobile/#/249201/',
 'https://www.yogiyo.co.kr/mobile/#/427615/',
 'https://www.yogiyo.co.kr/mobile/#/1083650/',
 'https://www.yogiyo.co.kr/mobile/#/1090048/',
 'https://www.yogiyo.co.kr/mobile/#/1227387/',
 'https://www.yogiyo.co.kr/mobile/#/1127009/',
 'https://www.yogiyo.co.kr/mobile/#/1088804/',
 'https://www.yogiyo.co.kr/mobile/#/236458/',
 'https://www.yogiyo.co.kr/mobile/#/1226657/',
 'https://www.yogiyo.co.kr/mobile/#/440654/',
 'https://www.yogiyo.co.kr/mobile/#/369210/',
 'https://www.yogiyo.co.kr/mobile/#/1297519/',
 'https://www.yogiyo.co.kr/mobile/#/363643/',
 'https://www.yogiyo.co.kr/mobile/#/551973/',
 'https://www.yogiyo.co.kr/mobile/#/1084391/',
 'https://www.yogiyo.co.kr/mobile/#/1209333/',
 'https://www.yogiyo.co.kr/mobile/#/1308196/',
 'https://www.yogiyo.co.kr/mobile/#/1028606/',
 'https://www.yogiyo.co.kr/mobile/#/39581/',
 'https://www.yogiyo.co.kr/mobile/#/1327097/',
 'https://www.yogiyo.co.kr/mobile/#/367184/',
 'https://www.yogiyo.co.kr/mobile/#/1272475/',
 'https://www.yogiyo.co.kr/mobile/#/225328/',
 'https://www.yogiyo.co.kr/mobile/#/429812/',
 'https://www.yogiyo.co.kr/mobile/#/1012990/',
 'https://www.yogiyo.co.kr/mobile/#/1084611/',
 'https://www.yogiyo.co.kr/mobile/#/1058650/',
 'https://www.yogiyo.co.kr/mobile/#/1296902/',
 'https://www.yogiyo.co.kr/mobile/#/191538/',
 'https://www.yogiyo.co.kr/mobile/#/1121937/',
 'https://www.yogiyo.co.kr/mobile/#/530838/',
 'https://www.yogiyo.co.kr/mobile/#/1150553/',
 'https://www.yogiyo.co.kr/mobile/#/1129703/',
 'https://www.yogiyo.co.kr/mobile/#/321571/',
 'https://www.yogiyo.co.kr/mobile/#/1237807/',
 'https://www.yogiyo.co.kr/mobile/#/1256793/',
 'https://www.yogiyo.co.kr/mobile/#/53115/',
 'https://www.yogiyo.co.kr/mobile/#/1328454/',
 'https://www.yogiyo.co.kr/mobile/#/1077962/',
 'https://www.yogiyo.co.kr/mobile/#/1213011/',
 'https://www.yogiyo.co.kr/mobile/#/1226877/',
 'https://www.yogiyo.co.kr/mobile/#/277246/',
 'https://www.yogiyo.co.kr/mobile/#/1152963/',
 'https://www.yogiyo.co.kr/mobile/#/1058474/',
 'https://www.yogiyo.co.kr/mobile/#/1225369/',
 'https://www.yogiyo.co.kr/mobile/#/547154/',
 'https://www.yogiyo.co.kr/mobile/#/39705/',
 'https://www.yogiyo.co.kr/mobile/#/1118052/',
 'https://www.yogiyo.co.kr/mobile/#/1281749/',
 'https://www.yogiyo.co.kr/mobile/#/1181542/',
 'https://www.yogiyo.co.kr/mobile/#/312917/',
 'https://www.yogiyo.co.kr/mobile/#/283174/',
 'https://www.yogiyo.co.kr/mobile/#/489424/',
 'https://www.yogiyo.co.kr/mobile/#/81574/',
 'https://www.yogiyo.co.kr/mobile/#/1311408/',
 'https://www.yogiyo.co.kr/mobile/#/1286516/',
 'https://www.yogiyo.co.kr/mobile/#/1216447/',
 'https://www.yogiyo.co.kr/mobile/#/1205890/',
 'https://www.yogiyo.co.kr/mobile/#/531022/',
 'https://www.yogiyo.co.kr/mobile/#/343174/',
 'https://www.yogiyo.co.kr/mobile/#/265690/',
 'https://www.yogiyo.co.kr/mobile/#/1001657/',
 'https://www.yogiyo.co.kr/mobile/#/1246990/',
 'https://www.yogiyo.co.kr/mobile/#/379790/',
 'https://www.yogiyo.co.kr/mobile/#/1234333/',
 'https://www.yogiyo.co.kr/mobile/#/1205780/',
 'https://www.yogiyo.co.kr/mobile/#/414470/',
 'https://www.yogiyo.co.kr/mobile/#/1299715/',
 'https://www.yogiyo.co.kr/mobile/#/1237700/',
 'https://www.yogiyo.co.kr/mobile/#/1257834/',
 'https://www.yogiyo.co.kr/mobile/#/1192423/',
 'https://www.yogiyo.co.kr/mobile/#/1312042/',
 'https://www.yogiyo.co.kr/mobile/#/213279/',
 'https://www.yogiyo.co.kr/mobile/#/354251/',
 'https://www.yogiyo.co.kr/mobile/#/262171/',
 'https://www.yogiyo.co.kr/mobile/#/1077861/',
 'https://www.yogiyo.co.kr/mobile/#/1238491/',
 'https://www.yogiyo.co.kr/mobile/#/1329792/',
 'https://www.yogiyo.co.kr/mobile/#/234922/',
 'https://www.yogiyo.co.kr/mobile/#/2328/',
 'https://www.yogiyo.co.kr/mobile/#/484834/',
 'https://www.yogiyo.co.kr/mobile/#/1209386/',
 'https://www.yogiyo.co.kr/mobile/#/1006476/',
 'https://www.yogiyo.co.kr/mobile/#/278272/',
 'https://www.yogiyo.co.kr/mobile/#/1236017/',
 'https://www.yogiyo.co.kr/mobile/#/311440/',
 'https://www.yogiyo.co.kr/mobile/#/1257066/',
 'https://www.yogiyo.co.kr/mobile/#/1293232/',
 'https://www.yogiyo.co.kr/mobile/#/1198710/',
 'https://www.yogiyo.co.kr/mobile/#/1143489/',
 'https://www.yogiyo.co.kr/mobile/#/39231/',
 'https://www.yogiyo.co.kr/mobile/#/1297230/',
 'https://www.yogiyo.co.kr/mobile/#/323388/',
 'https://www.yogiyo.co.kr/mobile/#/1293248/',
 'https://www.yogiyo.co.kr/mobile/#/1312779/',
 'https://www.yogiyo.co.kr/mobile/#/1301124/',
 'https://www.yogiyo.co.kr/mobile/#/1262047/',
 'https://www.yogiyo.co.kr/mobile/#/1241827/',
 'https://www.yogiyo.co.kr/mobile/#/379792/',
 'https://www.yogiyo.co.kr/mobile/#/292006/',
 'https://www.yogiyo.co.kr/mobile/#/1239133/',
 'https://www.yogiyo.co.kr/mobile/#/1263132/',
 'https://www.yogiyo.co.kr/mobile/#/1063104/',
 'https://www.yogiyo.co.kr/mobile/#/1271723/',
 'https://www.yogiyo.co.kr/mobile/#/1020970/',
 'https://www.yogiyo.co.kr/mobile/#/1265555/',
 'https://www.yogiyo.co.kr/mobile/#/1231562/',
 'https://www.yogiyo.co.kr/mobile/#/1241590/',
 'https://www.yogiyo.co.kr/mobile/#/369175/',
 'https://www.yogiyo.co.kr/mobile/#/1307769/',
 'https://www.yogiyo.co.kr/mobile/#/1270244/',
 'https://www.yogiyo.co.kr/mobile/#/363646/',
 'https://www.yogiyo.co.kr/mobile/#/1289249/',
 'https://www.yogiyo.co.kr/mobile/#/1271338/',
 'https://www.yogiyo.co.kr/mobile/#/1198487/',
 'https://www.yogiyo.co.kr/mobile/#/1167400/',
 'https://www.yogiyo.co.kr/mobile/#/1026462/',
 'https://www.yogiyo.co.kr/mobile/#/570729/',
 'https://www.yogiyo.co.kr/mobile/#/307444/',
 'https://www.yogiyo.co.kr/mobile/#/1310784/',
 'https://www.yogiyo.co.kr/mobile/#/1158531/',
 'https://www.yogiyo.co.kr/mobile/#/1285930/',
 'https://www.yogiyo.co.kr/mobile/#/418646/',
 'https://www.yogiyo.co.kr/mobile/#/427888/',
 'https://www.yogiyo.co.kr/mobile/#/374742/',
 'https://www.yogiyo.co.kr/mobile/#/1311961/',
 'https://www.yogiyo.co.kr/mobile/#/1106460/',
 'https://www.yogiyo.co.kr/mobile/#/1027064/',
 'https://www.yogiyo.co.kr/mobile/#/1297513/',
 'https://www.yogiyo.co.kr/mobile/#/267805/',
 'https://www.yogiyo.co.kr/mobile/#/1110735/',
 'https://www.yogiyo.co.kr/mobile/#/1242934/',
 'https://www.yogiyo.co.kr/mobile/#/1052684/',
 'https://www.yogiyo.co.kr/mobile/#/1050593/',
 'https://www.yogiyo.co.kr/mobile/#/1173561/',
 'https://www.yogiyo.co.kr/mobile/#/1293705/',
 'https://www.yogiyo.co.kr/mobile/#/1018007/',
 'https://www.yogiyo.co.kr/mobile/#/1201953/',
 'https://www.yogiyo.co.kr/mobile/#/1228074/',
 'https://www.yogiyo.co.kr/mobile/#/1296883/',
 'https://www.yogiyo.co.kr/mobile/#/332796/',
 'https://www.yogiyo.co.kr/mobile/#/1033767/',
 'https://www.yogiyo.co.kr/mobile/#/1333158/',
 'https://www.yogiyo.co.kr/mobile/#/1254416/',
 'https://www.yogiyo.co.kr/mobile/#/391531/',
 'https://www.yogiyo.co.kr/mobile/#/1309004/',
 'https://www.yogiyo.co.kr/mobile/#/1245211/',
 'https://www.yogiyo.co.kr/mobile/#/1297034/',
 'https://www.yogiyo.co.kr/mobile/#/468369/',
 'https://www.yogiyo.co.kr/mobile/#/1180586/',
 'https://www.yogiyo.co.kr/mobile/#/1271437/',
 'https://www.yogiyo.co.kr/mobile/#/84257/',
 'https://www.yogiyo.co.kr/mobile/#/1290167/',
 'https://www.yogiyo.co.kr/mobile/#/1211440/',
 'https://www.yogiyo.co.kr/mobile/#/1059627/',
 'https://www.yogiyo.co.kr/mobile/#/1256820/',
 'https://www.yogiyo.co.kr/mobile/#/1175565/',
 'https://www.yogiyo.co.kr/mobile/#/494696/',
 'https://www.yogiyo.co.kr/mobile/#/495264/',
 'https://www.yogiyo.co.kr/mobile/#/1321939/',
 'https://www.yogiyo.co.kr/mobile/#/1011734/',
 'https://www.yogiyo.co.kr/mobile/#/438470/',
 'https://www.yogiyo.co.kr/mobile/#/224001/',
 'https://www.yogiyo.co.kr/mobile/#/476215/',
 'https://www.yogiyo.co.kr/mobile/#/1270191/',
 'https://www.yogiyo.co.kr/mobile/#/1320640/',
 'https://www.yogiyo.co.kr/mobile/#/1074743/',
 'https://www.yogiyo.co.kr/mobile/#/439580/',
 'https://www.yogiyo.co.kr/mobile/#/1298805/',
 'https://www.yogiyo.co.kr/mobile/#/1040970/',
 'https://www.yogiyo.co.kr/mobile/#/318426/',
 'https://www.yogiyo.co.kr/mobile/#/284135/',
 'https://www.yogiyo.co.kr/mobile/#/216302/',
 'https://www.yogiyo.co.kr/mobile/#/1291392/',
 'https://www.yogiyo.co.kr/mobile/#/1175288/',
 'https://www.yogiyo.co.kr/mobile/#/560857/',
 'https://www.yogiyo.co.kr/mobile/#/1195936/',
 'https://www.yogiyo.co.kr/mobile/#/1280738/',
 'https://www.yogiyo.co.kr/mobile/#/1280737/',
 'https://www.yogiyo.co.kr/mobile/#/1081478/',
 'https://www.yogiyo.co.kr/mobile/#/1085329/',
 'https://www.yogiyo.co.kr/mobile/#/1041655/',
 'https://www.yogiyo.co.kr/mobile/#/227716/',
 'https://www.yogiyo.co.kr/mobile/#/288156/',
 'https://www.yogiyo.co.kr/mobile/#/567572/',
 'https://www.yogiyo.co.kr/mobile/#/1240421/',
 'https://www.yogiyo.co.kr/mobile/#/1201855/',
 'https://www.yogiyo.co.kr/mobile/#/1080857/',
 'https://www.yogiyo.co.kr/mobile/#/275915/',
 'https://www.yogiyo.co.kr/mobile/#/533022/',
 'https://www.yogiyo.co.kr/mobile/#/1301891/',
 'https://www.yogiyo.co.kr/mobile/#/1023684/',
 'https://www.yogiyo.co.kr/mobile/#/1221491/',
 'https://www.yogiyo.co.kr/mobile/#/302801/',
 'https://www.yogiyo.co.kr/mobile/#/63397/',
 'https://www.yogiyo.co.kr/mobile/#/1221810/',
 'https://www.yogiyo.co.kr/mobile/#/528520/',
 'https://www.yogiyo.co.kr/mobile/#/1041694/',
 'https://www.yogiyo.co.kr/mobile/#/1027900/',
 'https://www.yogiyo.co.kr/mobile/#/1216904/',
 'https://www.yogiyo.co.kr/mobile/#/1211923/',
 'https://www.yogiyo.co.kr/mobile/#/363645/',
 'https://www.yogiyo.co.kr/mobile/#/1239689/',
 'https://www.yogiyo.co.kr/mobile/#/9444/',
 'https://www.yogiyo.co.kr/mobile/#/397595/',
 'https://www.yogiyo.co.kr/mobile/#/370518/',
 'https://www.yogiyo.co.kr/mobile/#/1021679/',
 'https://www.yogiyo.co.kr/mobile/#/1196179/',
 'https://www.yogiyo.co.kr/mobile/#/370520/',
 'https://www.yogiyo.co.kr/mobile/#/1174073/',
 'https://www.yogiyo.co.kr/mobile/#/336726/',
 'https://www.yogiyo.co.kr/mobile/#/368743/',
 'https://www.yogiyo.co.kr/mobile/#/1115661/',
 'https://www.yogiyo.co.kr/mobile/#/1193943/',
 'https://www.yogiyo.co.kr/mobile/#/1145048/',
 'https://www.yogiyo.co.kr/mobile/#/1124915/',
 'https://www.yogiyo.co.kr/mobile/#/140865/',
 'https://www.yogiyo.co.kr/mobile/#/565255/']

In [235]:
review_data = []
failed_urls = []

for url in remained_list:
    try:
        driver.get(url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'lxml')

        # 가게 기본 데이터
        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

        # 리뷰 데이터
        review_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
        review_btn.click()
        time.sleep(1)

        def collect_reviews():
            soup = BeautifulSoup(driver.page_source, 'lxml')
            review_elements = soup.find_all('li', class_="list-group-item star-point ng-scope")
            for review_element in review_elements:
                score = review_element.find('span', class_='category')
                categorical_score = score.find_all('span', class_='points ng-binding')
                taste_score = categorical_score[0].text.strip()
                quantity_score = categorical_score[1].text.strip()
                review_time = review_element.find('span', class_='review-time ng-binding').text.strip()
                delivery_score = categorical_score[2].text.strip() if len(categorical_score) > 2 else None
                review_content = review_element.find('p', class_='ng-binding').text.strip()

                review_data.append({
                    'store_name': store_name,
                    'address': address,
                    'taste_score': taste_score,
                    'quantity_score': quantity_score,
                    'delivery_score': delivery_score,
                    'review': review_content,
                    'review_time': review_time
                })

        collect_reviews()

        # 더보기 버튼 클릭 함수
        def click_more_button():
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                plus_btn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, plus_btn_xpath))
                )
                plus_btn.click()
                return True
            except:
                return False

        i = 0
        while True:
            i += 1
            var = i * 10 + 2
            plus_btn_xpath = f'/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[{var}]/a'
            collect_reviews()

            if not click_more_button():
                break

            if i % 50 == 0:
                print(f"{i*10}번째 리뷰 크롤링 중...")

            time.sleep(1.5)  # 클릭 후 대기 시간 추가

    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        failed_urls.append(url)
        continue  # 다음 URL로 넘어갑니다.

review_df = pd.DataFrame(review_data).drop_duplicates().reset_index(drop=True)

# 실패한 URL이 있을 경우, 확인 가능
print("Failed URLs:", failed_urls)


500번째 리뷰 크롤링 중...
1000번째 리뷰 크롤링 중...


In [ ]:
main = pd.read_csv('dataset/review_data_0406.csv', index_col=0)
main = pd.concat([main, review_df], axis=0).reset_index(drop=True)
main.to_csv('dataset/review_data_0406.csv')
main